# Advent of Code 2025 - Day 8 (Nearest Neighbors + Union-Find)

In [31]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Test data
test = """162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689"""

# Parse test points
test_pts = [x.split(",") for x in test.split("\n")]
test_pts = np.array([[int(x) for x in y] for y in test_pts])
print(len(test_pts))

20


## Part 1

In [34]:
# Test example
pts = test_pts.copy()
K = 10
N = len(pts)

# Build nearest-neighbor graph using sklearn
# This efficiently finds all pairwise distances
nbrs = NearestNeighbors(n_neighbors=N, metric='euclidean').fit(pts)
dist, idx = nbrs.kneighbors(pts)

# Collect all unique edges (i < j to avoid duplicates)
edges = [(dist[i][j], i, idx[i][j])
         for i in range(N)
         for j in range(1, N)
         if i < idx[i][j]]

edges.sort(key=lambda x: x[0])  # Sort by distance (closest first)

# Union-Find data structure for tracking connected components
parent = list(range(N))
size = [1] * N

def find(x):
    """Find root with path compression"""
    while parent[x] != x:
        parent[x] = parent[parent[x]]  # Path compression
        x = parent[x]
    return x

def union(a, b):
    """Union two components using union by size"""
    ra, rb = find(a), find(b)
    if ra == rb:
        return
    if size[ra] < size[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    size[ra] += size[rb]

# Add the K shortest edges to connect points
for _, i, j in edges[:K]:
    union(i, j)

# Count circuit sizes (connected components)
counts = {}
for i in range(N):
    r = find(i)
    counts[r] = counts.get(r, 0) + 1

sizes = sorted(counts.values(), reverse=True)
answer = sizes[0] * sizes[1] * sizes[2]

print(f"Test result: {answer}")

Test result: 40


In [35]:
with open('input.txt', 'r') as f:
    data = f.read().strip()

pts = [x.split(",") for x in data.split("\n")]
pts = np.array([[int(x) for x in y] for y in pts])

K = 1000
N = len(pts)

# Build nearest-neighbor graph
nbrs = NearestNeighbors(n_neighbors=N, metric='euclidean').fit(pts)
dist, idx = nbrs.kneighbors(pts)

# Collect all unique edges
edges = [(dist[i][j], i, idx[i][j])
         for i in range(N)
         for j in range(1, N)
         if i < idx[i][j]]

edges.sort(key=lambda x: x[0])

# Union-Find
parent = list(range(N))
size = [1] * N

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra == rb:
        return
    if size[ra] < size[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    size[ra] += size[rb]

# Add the K shortest edges
for _, i, j in edges[:K]:
    union(i, j)

# Count circuit sizes
counts = {}
for i in range(N):
    r = find(i)
    counts[r] = counts.get(r, 0) + 1

sizes = sorted(counts.values(), reverse=True)
answer = sizes[0] * sizes[1] * sizes[2]

print(f"Part 1 answer: {answer}")

Part 1 answer: 153328


## Part 2

In [36]:
# Test example
pts = test_pts.copy()
N = len(pts)

# Build nearest-neighbor graph
nbrs = NearestNeighbors(n_neighbors=N, metric='euclidean').fit(pts)
dist, idx = nbrs.kneighbors(pts)

# Collect all unique edges
edges = [(dist[i][j], i, idx[i][j])
         for i in range(N)
         for j in range(1, N)
         if i < idx[i][j]]

edges.sort(key=lambda x: x[0])

# Union-Find
parent = list(range(N))
size = [1] * N

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra == rb:
        return False
    if size[ra] < size[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    size[ra] += size[rb]
    return True

def count_circuits():
    """Count how many distinct circuits exist"""
    roots = set()
    for i in range(N):
        roots.add(find(i))
    return len(roots)

# Add edges until only 2 circuits remain
edge_idx = 0
while count_circuits() > 2:
    if edge_idx >= len(edges):
        break
    _, i, j = edges[edge_idx]
    union(i, j)
    edge_idx += 1

# Get the two remaining circuits
circuits = {}
for i in range(N):
    r = find(i)
    if r not in circuits:
        circuits[r] = []
    circuits[r].append(i)

# Sort circuits by size and get the two largest
circuit_list = sorted(circuits.items(), key=lambda x: len(x[1]), reverse=True)
junction_box_1 = np.array([pts[i] for i in circuit_list[0][1]])
junction_box_2 = np.array([pts[i] for i in circuit_list[1][1]])

# Find closest point between the two junction boxes
min_dist = float('inf')
closest_point_in_box1 = None
closest_point_in_box2 = None

for p1 in junction_box_1:
    for p2 in junction_box_2:
        dist = np.linalg.norm(p1 - p2)
        if dist < min_dist:
            min_dist = dist
            closest_point_in_box1 = p1
            closest_point_in_box2 = p2

answer = int(closest_point_in_box1[0]) * int(closest_point_in_box2[0])
print(f"Test result: {answer}")

Test result: 25272


In [37]:
with open('input.txt', 'r') as f:
    data = f.read().strip()

pts = [x.split(",") for x in data.split("\n")]
pts = np.array([[int(x) for x in y] for y in pts])

N = len(pts)

# Build nearest-neighbor graph
nbrs = NearestNeighbors(n_neighbors=N, metric='euclidean').fit(pts)
dist, idx = nbrs.kneighbors(pts)

# Collect all unique edges
edges = [(dist[i][j], i, idx[i][j])
         for i in range(N)
         for j in range(1, N)
         if i < idx[i][j]]

edges.sort(key=lambda x: x[0])

# Union-Find
parent = list(range(N))
size = [1] * N

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra == rb:
        return False
    if size[ra] < size[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    size[ra] += size[rb]
    return True

def count_circuits():
    """Count how many distinct circuits exist"""
    roots = set()
    for i in range(N):
        roots.add(find(i))
    return len(roots)

# Add edges until only 2 circuits remain
edge_idx = 0
while count_circuits() > 2:
    if edge_idx >= len(edges):
        break
    _, i, j = edges[edge_idx]
    union(i, j)
    edge_idx += 1

# Get the two remaining circuits
circuits = {}
for i in range(N):
    r = find(i)
    if r not in circuits:
        circuits[r] = []
    circuits[r].append(i)

# Sort circuits by size and get the two largest
circuit_list = sorted(circuits.items(), key=lambda x: len(x[1]), reverse=True)
junction_box_1 = np.array([pts[i] for i in circuit_list[0][1]])
junction_box_2 = np.array([pts[i] for i in circuit_list[1][1]])

# Find closest point between the two junction boxes
min_dist = float('inf')
closest_point_in_box1 = None
closest_point_in_box2 = None

for p1 in junction_box_1:
    for p2 in junction_box_2:
        dist = np.linalg.norm(p1 - p2)
        if dist < min_dist:
            min_dist = dist
            closest_point_in_box1 = p1
            closest_point_in_box2 = p2

answer = int(closest_point_in_box1[0]) * int(closest_point_in_box2[0])
print(f"Part 2 answer: {answer}")


Part 2 answer: 6095621910
